In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import asyncio
import aiohttp
import nest_asyncio
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
from loguru import logger
from datetime import datetime
import glob
from matplotlib import pyplot as plt
from collections import Counter
import time
from fake_useragent import UserAgent
from collections import Counter
import urllib

from urllib.request import Request, urlopen

nest_asyncio.apply()


In [ ]:
base_url = 'www.bookyogaretreats.com'

# url='https://www.bookyogaretreats.com/yoga-weeks/6-day-urban-yoga-retreat-in-barcelona-catalonia'
url='https://www.bookyogaretreats.com/?page=504'
print(url)

# r = requests.get(url)
# response = r.text.encode('utf-8')

r = Request(url , headers={'User-Agent': 'Mozilla/5.0'})
response = urlopen(r).read()

soup = BeautifulSoup(response, 'html.parser')
soup
# soup.findAll('a', {'class': 'js-showcard-link'}, href=True)[0]['href']


In [ ]:
retreat_links = []

pages_number = 504
workers = 20

pages_step = int(pages_number / workers)
diff = pages_number - pages_step * workers

pages_start = [pages_step * i + 1 for i in range(workers)]
pages_end = [pages_step * i for i in range(1, workers + 1)]
pages_end[-1] += diff

page_category = 'other-retreats'

async def get_links(start, end):
    global retreat_links
    for page in range(start, end + 1):
        print(page)
        url = f'https://www.bookyogaretreats.com/?page={page}'

        async with aiohttp.ClientSession() as session:
            async with session.get(url,headers={'user-agent': 'Mozilla/5.0'}) as response:
                response = await response.text()
                soup = BeautifulSoup(response, 'html.parser')

                links = soup.findAll('a', {'class': 'js-showcard-link'}, href=True)
                for link in links:
                    retreat_link = 'https://www.bookyogaretreats.com' + link['href']
                    retreat_links.append(retreat_link)
                np.save('bookyogaretreats_links.npy', retreat_links)


futures = [get_links(pages_start[i], pages_end[i]) for i in range(workers)]

loop = asyncio.get_event_loop()
loop.run_until_complete(asyncio.wait(futures))
# loop.close()

In [ ]:
arr=np.load('bookyogaretreats_links.npy')
arr.shape

In [ ]:
# a1= np.load('meditation-retreats_retreats_links.npy')
# a2 = np.load('other-retreats_retreats_links.npy')
# a3 = np.load('ttc_retreats_links.npy')
# a4 = np.load('wellness-retreats_retreats_links.npy')
# a5 = np.load('yoga-retreats_retreats_links.npy')
# links=np.concatenate([a1,a2,a3,a4,a5])

In [ ]:
retreats = pd.DataFrame(
    {'title': [0], 'duration': [0], 'center_title': [0], 'center_url': [0], 'location': [0], 'country': [0],
     'category': [0], 'price': [0],
     'deposit': [0], 'retreat_url': [0]})
retreats.to_csv('retreats_bookretreats.csv', encoding='utf-8', index=False)

In [ ]:
retreats  = pd.read_csv('retreats_bookretreats.csv')
retreats.head(5)

In [ ]:
retreats.info()

In [ ]:
links=np.load('retreat_links_bookretreat.npy')

base_url = 'https://bookretreats.com'

pages_number = 2339
workers = 15
folder = 'centres_images'
# img_links = []

pages_step = int(pages_number / workers)
diff = pages_number - pages_step * workers

pages_start = [pages_step * i + 1 for i in range(workers)]
pages_end = [pages_step * i for i in range(1, workers + 1)]
pages_end[-1] += diff - 1



async def get_retreat(start, end):
    global retreats
    # global img_links
    global links
    for i in range(start, end + 1):
        print(i)
        retreat_url = links[i]

        async with aiohttp.ClientSession() as retreat_session:
            async with retreat_session.get(retreat_url) as retreat_response:

                retreat_response = await retreat_response.text()
                soup = BeautifulSoup(retreat_response, 'html.parser')


                title = soup.findAll('div', {'class': 'col-span-4 sm:col-span-3 flex items-center'})[0].text


                duration =soup.findAll('div', {'class': 'mb-4 text-base text-gray-500 dark:text-gray-300'})[0].text.split(' ')[0]


                loc=soup.findAll('a', {'class': 'block text-sm leading-7 text-gray-500 dark:text-gray-400'})
                country=loc[-2].text
                location=country+' '+loc[-1].text
                category=loc[1].text

                deposit = soup.findAll('div', {'class': 'tabular-nums text-xl text-gray-500 dark:text-gray-200 font-semibold'})[
                              0].text[1:].replace(',', '')[3:]

                price = soup.findAll('div', {'class': 'tabular-nums text-2xl text-gray-500 dark:text-gray-200 font-extrabold'})[
                            0].text[1:].replace(',', '')[3:]

                center = soup.find('a', {'class': 'block text-gray-900 dark:text-gray-200 text-lg font-bold mb-2'})
                center_title = center.text
                center_half_url = center['href']
                center_url = base_url + center_half_url

                retreats  = retreats.append(
                    {'title': title, 'duration': duration, 'center_title': center_title, 'center_url': center_url,
                     'location': location, 'country': country, 'category': category,
                     'price': price, 'deposit': deposit, 'retreat_url': retreat_url}, ignore_index=True)
                retreats.to_csv('retreats_bookretreats.csv', encoding='utf-8', index=False)

futures = [get_retreat(pages_start[i], pages_end[i]) for i in range(workers)]

loop = asyncio.get_event_loop()
loop.run_until_complete(asyncio.wait(futures))